In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import requests

proxy_ip=!curl "http://metadata.google.internal/computeMetadata/v1/instance/attributes/PROXY_IP" -H "Metadata-Flavor: Google"
proxy_ip=(proxy_ip[5])
proxies = {'https': proxy_ip}
print("Secure web proxy address is {}".format(proxies))

In [ ]:
# Whitelisted domains in secure web proxy:
#    github.com
#    pypi.org
#    pypi.python.org
#    files.pythonhosted.org
#    packaging.python.org
#    cloud.r-project.org

In [ ]:
# Authorized domain/URL request witll have a 200 code response
whitelisted_url = requests.get('https://github.com/GCP-Architecture-Guides/csa-user-managed-vertex-AI-workbench', proxies=proxies)
print(f'Status Code: {whitelisted_url.status_code}')

In [ ]:
# Unauthorized domain/URL request will blocked and timed out.
unauthorized_url = requests.get('http://cloud.google.com/robots.txt', proxies=proxies)
print(f'Status Code: {unauthorized_url.status_code}')

In [ ]:
# This demo is not configured to perform TLS inspection, hence using "--trusted-host" while using proxy to download packages.
! pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pip setuptools  --proxy "http://{proxy_ip}"

In [ ]:
# This demo is not configured to perform TLS inspection, hence using "--trusted-host" while using proxy to download packages.
! pip3 install --trusted-host --upgrade  google-cloud-aiplatform --proxy "http://{proxy_ip}"

In [ ]:
# Making an internal API call to get Project_ID
project_id=!curl "http://metadata.google.internal/computeMetadata/v1/instance/attributes/PROJ_ID" -H "Metadata-Flavor: Google"
project_id=(project_id[5])
print("Vertex project id: {}".format(project_id))

In [ ]:
# Making an internal API call to get Data Project_ID
data_project_id=!curl "http://metadata.google.internal/computeMetadata/v1/instance/attributes/DATA_PROJ_ID" -H "Metadata-Flavor: Google"
data_project_id=(data_project_id[5])
print("Data project id: {}".format(data_project_id))

In [ ]:
# Making an internal API call to get storage bucket name
bucket_name=!curl "http://metadata.google.internal/computeMetadata/v1/instance/attributes/DATA_BUCKET" -H "Metadata-Flavor: Google"
bucket_name=(bucket_name[5])
print("GCS storage bucket name: {}".format(bucket_name))

In [ ]:
# List the bucket data files from data project
!gsutil ls gs://{bucket_name}

In [ ]:
!gsutil cat -h -r 0-1000 gs://{bucket_name}/*

In [ ]:
# Making an internal API call to get BigQuery dataset ID
dataset_id=!curl "http://metadata.google.internal/computeMetadata/v1/instance/attributes/DATASET" -H "Metadata-Flavor: Google"
datasett=(dataset_id[5])
print("BigQuery data set id: {}".format(datasett))

In [8]:
# Access the BigQuery dataset from data project
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set dataset_id to the ID of the dataset to fetch.
dataset_id = data_project_id + "." + datasett

dataset = client.get_dataset(dataset_id)  # Make an API request.

full_dataset_id = "{}.{}".format(dataset.project, dataset.dataset_id)

# View dataset properties.
print("Description: {}".format(dataset.description))
print("Labels:")
labels = dataset.labels
if labels:
    for label, value in labels.items():
        print("\t{}: {}".format(label, value))
else:
    print("\tDataset has no labels defined.")

# View tables in dataset.
print("Tables:")
tables = list(client.list_tables(dataset))  # Make an API request(s).
if tables:
    for table in tables:
        print("\t{}".format(table.table_id))
        table_id = table.table_id
else:
    print("\tThis dataset does not contain any tables.")

In [ ]:
full_table_id = "{}:{}.{}".format(data_project_id, datasett, table_id)
print("!bq show {}".format(full_table_id))